In [ ]:
import pandas as pd
import geopandas
import folium 
import requests
import glob
import arrow
from collections import Counter
from folium import plugins
import arrow

In [ ]:
list_files = glob.glob(r"..\data\\raw_tfl_data\*.csv")
list_files

In [ ]:
interim_container = []

for _file in list_files: 
    
    _data = pd.read_csv(_file)
    interim_container.append(_data)

combined_data = interim_container[0].append(interim_container[1:])

In [ ]:
combined_data = combined_data.rename(columns = {"Start Date":"Start_Date", "End Date":"End_Date"})

In [ ]:
combined_data["Start_Date_Converted"] = pd.to_datetime(combined_data["Start_Date"], format = "%d/%m/%Y %H:%M")
combined_data["End_Date_Converted"] = pd.to_datetime(combined_data["End_Date"], format = "%d/%m/%Y %H:%M")

In [ ]:
combined_data = combined_data.sort_values(by="Start_Date_Converted")

In [ ]:
combined_data.shape

In [ ]:
def data_request(station_name): 
    api_key = ""
    r = requests.get(f"https://api.opencagedata.com/geocode/v1/json?q={station_name}&key={api_key}")
    return r.json()




In [ ]:
start_station = (combined_data["StartStation Name"].to_list())
end_station = (combined_data["EndStation Name"].to_list())
combined_list_of_stations = start_station + end_station
all_stations = Counter(combined_list_of_stations)


In [ ]:
lat_dict = {}
lng_dict ={}

for key in all_stations.keys(): 

    data = data_request(key)

    try:
        
        if data["results"][0]["components"]["city"] == "London":
            
            print (data["results"][0])
            lat = data["results"][0]["geometry"]["lat"]
            lng = data["results"][0]["geometry"]["lng"]
        
        else: 
            
            lat = None 
            lng = None 
        
    except KeyError: 
        
        lat = None 
        lng = None
        
    except IndexError: 
        
        lat = None 
        lng = None        
    
    else: 
        
        lat_dict[key] = lat
        lng_dict[key] = lng
        
    print (len(lat_dict))

In [ ]:
combined_data['start_lat'] = combined_data['StartStation Name'].map(lat_dict)
combined_data['end_lat']= combined_data['EndStation Name'].map(lat_dict)
combined_data['start_lng'] = combined_data['StartStation Name'].map(lng_dict)
combined_data['end_lng']= combined_data['EndStation Name'].map(lng_dict)

In [ ]:
combined_data

In [ ]:
combined_data_filtered = combined_data[combined_data["Bike Id"] == 7753]
#combined_data_filtered = combined_data_filtered[combined_data_filtered["Start_Date"].str.contains("/02/2020")]
combined_data_filtered = combined_data_filtered.dropna()
combined_data_filtered = combined_data_filtered.reset_index()

In [ ]:
columns = list(combined_data_filtered.columns)
coordinates = []
times = []
lines = []

for row in combined_data_filtered.itertuples(): 
    
    start = [row[columns.index('start_lng')+1], row[columns.index('start_lat')+1]]
    end = [row[columns.index('end_lng')+1], row[columns.index('end_lat')+1]]

    time_start = arrow.get(row[columns.index('Start_Date_Converted')+1]).isoformat().replace('+00:00', '')
    time_end = arrow.get(row[columns.index('End_Date_Converted')+1]).isoformat().replace('+00:00', '')
    
    lines.append({'coordinates': [start, end],'dates': [time_start,time_end]})

In [ ]:
m = folium.Map(
    location=[51.509865, -0.118092],
    zoom_start=12,
    tiles='openstreet map')


for _, row in combined_data_filtered.iterrows():
    
    tooltip = _
    
    
    try: 
        folium.CircleMarker([row['start_lat'], row['start_lng']],
                            radius=15,
                            fill_color="#3db7e4", # divvy color
                            tooltip=tooltip
                           ).add_to(m)

        folium.CircleMarker([row['end_lat'], row['end_lng']],
                            radius=15,
                            fill_color="red", # divvy color
                            tooltip=tooltip
                           ).add_to(m)

#         folium.PolyLine([[row['start_lat'], row['start_lng']], 
#                          [row['end_lat'], row['end_lng']]]).add_to(m)
        
    except ValueError: 
        pass
m

In [ ]:
combined_data_filtered.reset_index()

In [ ]:
m = folium.Map(tiles='cartodbpositron', location=(51.51, -0.1), zoom_start=12)


features = [{'type': 'Feature',
             'geometry': {'type': 'LineString',
                    'coordinates': line['coordinates'],},
              'properties': {'times': line['dates'],
                            'style': {'weight': line['weight'] if 'weight' in line else 5}}}
            for line in lines]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='P1D',  time_slider_drag_update=True, add_last_point=True).add_to(m)

In [ ]:
m.save("TheJourneyOfBike7753.html")

In [ ]:
start_dates = combined_data_filtered.Start_Date.to_list()

formatted = [arrow.get(start_date, 'DD/MM/YYYY HH:mm').format("DD/MM/YYYY") for start_date in start_dates]

In [ ]:
counts = Counter(formatted)

In [ ]:
counter_dataframe = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counter_dataframe = counter_dataframe.rename(columns = {"index":"Journey Start Dates", 0:"Total"})

In [ ]:
counter_dataframe.plot.line(x="Journey Start Dates", y="Total", figsize = [10,5])